In [1]:
from datasets import load_dataset, Dataset
import pandas as pd
ds = load_dataset("HuggingFaceH4/ultrafeedback_binarized", split="train_prefs")


/home/lidong1/miniconda3/envs/llama_factory/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = ds.to_pandas()
df['chosen'][0][0]['content']

'how can i develop a habit of drawing daily'

In [3]:
def get_instruct_response(item):
    # item["instruction"] = item['chosen'][0]["content"]
    item["response"] = item['chosen'][1]["content"]
    return item
ds = ds.map(get_instruct_response, batch_size=1024, num_proc=8)
export_data = ds.select_columns(["prompt", "response"])
export_data

Dataset({
    features: ['prompt', 'response'],
    num_rows: 61135
})

In [18]:
from gpt4o import Openai, API_INFOS
client = Openai(apis=API_INFOS)
ans = client.get_response("你是gpt几？", system="you are gpt2.5, a helpful chat assisstant")
print(ans)

我是GPT-2.5，一个帮助你回答问题和提供信息的聊天助手。有什么我可以帮你的吗？


我需要写一个prompt，让gpt帮我将LLM对于给定instruction生成的文本在文本美学方面进行修改的prompt，以下是我写的一个中文的system_template，对于这个template，如何让gpt更好的完成我的任务，给出修改意见和修改后的英语的system_template
system_template = \
"""请扮演一个文本改写者来改写LLM(Large Language Model)生成的文本，使其更具有文本美学。你的任务是对于提供的用户instruction和一个LLM生成的答案，你从文本美学的角度去思考这个答案是否合理，是否需要修改,然后给出修改的文本。
*Format*
文本美学分析:
[文本美学分析]
是否需要修改:[Y/N]
修改后的文本：
[修改后的文本]
"""

我需要写一个prompt，让gpt帮我将LLM对于给定instruction生成的文本在文本美学方面进行修改的prompt，以下是我写的一个system template 和user template，对于这个template，如何让gpt更好的完成我的任务，给出修改意见和修改后的英语的template
system_template = \
"""You are tasked with acting as a text rewriter to enhance the aesthetic quality of text generated by a Large Language Model (LLM). Your job is to evaluate the provided user instruction and the LLM-generated response from an aesthetic perspective, determine whether the text needs modification, and then provide the revised text if necessary.   
  
Consider the following aspects of textual aesthetics:  
- **Fluency**: Is the text smooth and easy to read?  
- **Vocabulary**: Are the words used varied and appropriate?  
- **Sentence Structure**: Are the sentences well-constructed and rhythmically balanced?  
- **Imagery and Tone**: Does the text evoke vivid images and maintain a consistent tone?  
  
*Format*  
Textual Aesthetic Analysis: [Your analysis]  
Does it need modification: [Y/N]  
Revised Text: [Your revised text, if applicable]  
"""  

user_template = \
"""### User Instruction:
{instruction}

### Answer
{completition}

Your task is to:  
1. Analyze the LLM-generated response from a textual aesthetic perspective.  
2. Determine whether the text needs modification.  
3. Provide a revised version of the text if necessary.  
  
Consider the following aspects of textual aesthetics:  
- **Fluency**: Is the text smooth and easy to read?  
- **Vocabulary**: Are the words used varied and appropriate?  
- **Sentence Structure**: Are the sentences well-constructed and rhythmically balanced?  
- **Imagery and Tone**: Does the text evoke vivid images and maintain a consistent tone?  
  
*Format*  
Textual Aesthetic Analysis: [Your analysis]  
Does it need modification: [Y/N]  
Revised Text: [Your revised text, if applicable] 
"""

In [4]:
system_template = \
"""You are tasked with acting as a text rewriter to enhance the aesthetic quality of text generated by a Large Language Model (LLM). Your job is to evaluate the provided user instruction and the LLM-generated response from an aesthetic perspective, determine whether the text needs modification, and then provide the revised text if necessary.   
  
Consider the following aspects of textual aesthetics:  
- **Fluency**: Is the text smooth and easy to read?  
- **Vocabulary**: Are the words used varied and appropriate?  
- **Sentence Structure**: Are the sentences well-constructed and rhythmically balanced?  
- **Imagery and Tone**: Does the text evoke vivid images and maintain a consistent tone?  
  
*Format*  
Textual Aesthetic Analysis: [Your analysis]  
Does it need modification: [Y/N]  
Revised Text: [Your revised text, if applicable]  
"""  

In [5]:
user_template = \
"""### User Instruction:
{instruction}

### Answer
{completition}

Your task is to:  
1. Analyze the LLM-generated response from a textual aesthetic perspective.  
2. Determine whether the text needs modification.  
3. Provide a revised version of the text if necessary.  
  
Consider the following aspects of textual aesthetics:  
- **Fluency**: Is the text smooth and easy to read?  
- **Vocabulary**: Are the words used varied and appropriate?  
- **Sentence Structure**: Are the sentences well-constructed and rhythmically balanced?  
- **Imagery and Tone**: Does the text evoke vivid images and maintain a consistent tone?  
  
*Format*  
Textual Aesthetic Analysis: [Your analysis]  
Does it need modification: [Y/N]  
Revised Text: [Your revised text, if applicable] 
"""

In [6]:
from gpt4o import Openai, API_INFOS
client = Openai(apis=API_INFOS)
# ans = client.get_response("你是gpt几？", system="you are gpt2.5, a helpful chat assisstant")
# print(ans)
def get_revised_text(client, instruction, completition, user_template, system_template, max_tokens=2048):
    content = user_template.format(instruction=instruction, completition=completition)
    gpt_answer = client.get_response(content=content, system=system_template, max_tokens=max_tokens)
    gpt_answer = gpt_answer.strip()
    need_modification = "Y" if "Does it need modification: Y" in gpt_answer else "N"  
    revised_text_start = gpt_answer.find("Revised Text:") + len("Revised Text:")  
    revised_text = gpt_answer[revised_text_start:].strip() 
    return need_modification, revised_text, gpt_answer  

In [7]:
from tqdm import tqdm
revised_data = []  
sample_data = export_data.select(range(10))
for row in tqdm(sample_data):
    prompt = row['prompt']  
    response = row['response']

    need_modification, revised_text, gpt_answer = get_revised_text(client, prompt, response, user_template, system_template, max_tokens=2048) 
    # 将结果添加到列表  
    revised_data.append({  
        'prompt': prompt,  
        'response': response,  
        'does_it_need_modification': need_modification,  
        'revised_text': revised_text,
        'gpt_answer': gpt_answer 
    })  
# 创建新的Dataset  
revised_dataset = Dataset.from_pandas(pd.DataFrame(revised_data))  

100%|██████████| 10/10 [02:13<00:00, 13.33s/it]


In [9]:
revised_dataset.shape[0]

10

In [10]:
data_length = revised_dataset.shape[0]
revised_dataset.to_json(f"./revised_data/revised_{data_length}.jsonl")
revised_dataset

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 338.44ba/s]


Dataset({
    features: ['prompt', 'response', 'does_it_need_modification', 'revised_text', 'gpt_answer'],
    num_rows: 10
})

In [12]:
print(revised_data[1]['gpt_answer'])

Textual Aesthetic Analysis: The LLM-generated response is clear and provides practical advice. However, it could benefit from a more engaging tone and varied sentence structure. The vocabulary is appropriate, but the text could be more fluid and rhythmically balanced. Additionally, the response could be more concise while still conveying the necessary information.

Does it need modification: Y

Revised Text: Modifying built-in methods is generally discouraged as it can introduce unexpected issues and bugs. Instead, consider creating a new method or exploring alternative approaches to achieve your goal. You might also look into other libraries or modules that offer similar functionalities, or reach out to the library's support team for further guidance.


In [13]:
df = export_data.to_pandas()
df[df['prompt'] == 'Create a personalized workout plan for a client with a unique set of physical limitations and goals. Utilize a provided fitness template and provide modifications for each exercise to accommodate their limitations.\nName: Emily\nAge: 45\nLimitations: Arthritis in knees and shoulders\nGoals: Improve cardiovascular health and increase upper body strength\n<template>\nFitness Plan for Emily:\nCardiovascular Exercise:\n- ___________________ (modifications: ____________________)\nUpper Body Strength:\n- ___________________ (modifications: ____________________)\n- ___________________ (modifications: ____________________)\nRemember to warm up and cool down properly before and after each workout. If any exercises cause pain or discomfort, please discontinue and consult with a medical professional.","response":"Fitness Plan for Emily:\nCardiovascular Exercise:\n- Stationary Bike (modifications: adjust seat height and resistance to be gentle on knees, maintain a comfortable pace)\nUpper Body Strength:\n- Seated Dumbbell Shoulder Press (modifications: use light weights and perform slow, controlled movements)\n- Wall Push-ups (modifications: stand at a comfortable distance from the wall to reduce stress on shoulders, perform slow, controlled movements)\nRemember to warm up and cool down properly before and after each workout. If any exercises cause pain or discomfort, please discontinue and consult with a medical professional.']

,prompt,response


In [39]:
export_data.select(range(100))

Dataset({
    features: ['prompt', 'response'],
    num_rows: 100
})

In [40]:
export_data.select(range(100)).to_json("uf_sample.jsonl")

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 250.24ba/s]


216011